In [ ]:
# default_exp engine.point_process

In [ ]:
%load_ext autoreload
%autoreload 2

# Emitter distribution

> Definition of the class used to simulate random emitter positions and intensities

In [ ]:
#export
from decode_fish.imports import *
from torch import distributions as D, Tensor
from torch.distributions import Distribution

In [ ]:
#export
class PointProcessUniform(Distribution):
    """
    This class is part of the generative model and uses probability local_rate to generate locations `locations`  `x`, `y`, `z` offsets and `intensities` intensity of emitters. local_rate  should be `torch.tensor` scaled from 0.001 to 1), which is used by `_sample_bin` to generate `0` and `1` . `0` means that we don't have an emitter at a given pixel, and 1 means emitters is present. This map is used to generate offset in `x`, `y`, `z`, and intensities, which tells how bright is emitter or, in some cases, how many emitters are bound to given molecules.
    Args:
        local_rate (BS, C, H, W, D): Local rate
        min_int (int): minimum intensity of emitters
        bg(bool): if returns sampled backround

    """
    def __init__(self, local_rate: torch.tensor, min_int: float):

        self.local_rate = local_rate
        self.device = self._get_device(self.local_rate)
        self.min_int = torch.tensor(min_int, device=self.device)

    def sample(self, N:int =1):

        if N == 1: return self._sample()
        else:
            res_ = [self._sample() for i in range(N)]
            locations = torch.cat([i[0] for i in res_], dim=1)
            x_offset = torch.cat([i[1] for i in res_], dim=1)
            y_offset = torch.cat([i[2] for i in res_], dim=1)
            z_offset = torch.cat([i[3] for i in res_], dim=1)
            intensities = torch.cat([i[4] for i in res_], dim=1)
            return locations, x_offset, y_offset, z_offset, intensities

    def _sample(self):

        locations = D.Bernoulli(self.local_rate).sample()
        n_emitter = int(locations.sum().item())
        zero_point_five = torch.tensor(0.5, device=self.device)
        x_offset = D.Uniform(low=0 - zero_point_five, high=0 + zero_point_five).sample(sample_shape=[n_emitter])
        y_offset = D.Uniform(low=0 - zero_point_five, high=0 + zero_point_five).sample(sample_shape=[n_emitter])
        z_offset = D.Uniform(low=0 - zero_point_five, high=0 + zero_point_five).sample(sample_shape=[n_emitter])
        intensities = D.Uniform(low=self.min_int, high=1.0).sample(sample_shape=[n_emitter])
        
        output_shape = tuple(locations.shape)
        locations = locations.nonzero(as_tuple=True)
        return locations, x_offset, y_offset, z_offset, intensities, output_shape


    def log_prob(self, locations, x_offset=None, y_offset=None, z_offset=None, intensities=None, output_shape=None):
        locations = list_to_locations(locations, output_shape)
        log_prob = D.Bernoulli(self.local_rate).log_prob(locations)
        return log_prob

    @staticmethod
    def _get_device(x):
        return getattr(x, 'device')
    

def list_to_locations(locations, output_shape):
    tmp =torch.zeros(output_shape, device=locations[0].device)
    coord = torch.stack(locations).T
    #incase you have multiple emitter present
    for i in coord: tmp[tuple(i)] += 1
    return tmp

In [ ]:
point_process = PointProcessUniform(local_rate = torch.ones([1,1,48,48,48])*.01, min_int = 0.5)
locs_3d, x_os_3d, y_os_3d, z_os_3d, ints_3d, output_shape = point_process.sample()
len(locs_3d[0])

1102

In [ ]:
!nbdev_build_lib

Converted 00_models.ipynb.
Converted 01_psf.ipynb.
Converted 02_microscope.ipynb.
Converted 03_noise.ipynb.
Converted 04_pointsource.ipynb.
Converted 05_gmm_loss.ipynb.
Converted 06_plotting.ipynb.
Converted index.ipynb.
